In [1]:
import pandas as pd
import sqlite3
import os
import time


subject = 'IorNYll1lS' #! The ID of the subject on Back4app
folder_path = 'database extractor/data'
df_file = '20240412_102837_14ChannelSensorDatabase_mci011.db'
db_file_path = f'{folder_path}/{df_file}'


In [3]:
# Create a connection to the SQLite database
conn = sqlite3.connect(db_file_path)

start_session = 1
end_session = 125

# Write your SQL query (replace 'your_table_name' with the actual table name)
sql_query = f'SELECT * FROM USER_SESSION_TABLE WHERE SESSION_ID BETWEEN {start_session} AND {end_session}'

# Use pandas to read data from the database into a DataFrame
df = pd.read_sql(sql_query, conn)

# Close the database connection
conn.close()


In [4]:
start_time = time.time()
print("--- %s seconds ---" % (time.time() - start_time))

# Create a connection to the SQLite database
conn = sqlite3.connect(db_file_path)

# Write SQL queries for each table
right_insole_query = f'SELECT * FROM RIGHT_INSOLE_RAW WHERE SESSION_ID BETWEEN {start_session} AND {end_session}'
left_insole_query = f'SELECT * FROM LEFT_INSOLE_RAW WHERE SESSION_ID BETWEEN {start_session} AND {end_session}'

# Use pandas to read data from each table into DataFrames
right_insole_df = pd.read_sql(right_insole_query, conn)
print("--- Right Query %s seconds ---" % (time.time() - start_time))
left_insole_df = pd.read_sql(left_insole_query, conn)
print("--- Left Query %s seconds ---" % (time.time() - start_time))

# Close the database connection
conn.close()

# Group the DataFrames by SESSION_ID
grouped_right_insole = right_insole_df.groupby('SESSION_ID').agg(list).reset_index()
grouped_left_insole = left_insole_df.groupby('SESSION_ID').agg(list).reset_index()
print("--- Complete in %s seconds ---" % (time.time() - start_time))


--- 0.0 seconds ---
--- Right Query 53.72006106376648 seconds ---
--- Left Query 99.88784885406494 seconds ---
--- Complete in 120.76854586601257 seconds ---


In [5]:
import time
start_time = time.time()
print("--- %s seconds ---" % (time.time() - start_time))


# Create a connection to the SQLite database
conn = sqlite3.connect(db_file_path)

# Read USER_SESSION_TABLE to get STARTDATE for each SESSION_ID
user_session_query = 'SELECT SESSION_ID, START_DATE FROM USER_SESSION_TABLE'
user_session_df = pd.read_sql(user_session_query, conn)
print("--- Read %s seconds ---" % (time.time() - start_time))

# Merge user_session_df with grouped_right_insole and grouped_left_insole
merged_right_insole = pd.merge(grouped_right_insole, user_session_df, on='SESSION_ID', how='inner')
merged_left_insole = pd.merge(grouped_left_insole, user_session_df, on='SESSION_ID', how='inner')


print("--- Merged in %s seconds ---" % (time.time() - start_time))
# Close the database connection
conn.close()


# Root folder path
root_folder_path = 'database extractor/data/mci006/'

# Filter data for the specified SESSION_ID range
filtered_right_insole = merged_right_insole[(merged_right_insole['SESSION_ID'] >= start_session) & (merged_right_insole['SESSION_ID'] <= end_session)]
filtered_left_insole = merged_left_insole[(merged_left_insole['SESSION_ID'] >= start_session) & (merged_left_insole['SESSION_ID'] <= end_session)]

print("--- Filtered in %s seconds ---" % (time.time() - start_time))
# Iterate through each row and save data as TXT in the specified folder
for index, session in filtered_right_insole.iterrows():
    session_id = session['SESSION_ID']
    start_date = str(session['START_DATE'])
    date_folder = start_date[:4] + '-' + start_date[4:6] + '-' + start_date[6:]
    array_data = session.drop(['SESSION_ID', 'START_DATE', 'RAW_ID'])
    session_df = pd.DataFrame(array_data).T
    folder_path = os.path.join(root_folder_path, date_folder)
    
    # Convert each array to pandas Series
    series_list = {col: pd.Series(data) for col, data in array_data.items()}
    if len(series_list['READING_1']) < 2 / 0.05: #! data can be less than 2 seconds, which is not necessary
        print(f'Session {session_id} less than 2 seconds, Skip')
        print("-----------------------------------------------------------")
        continue
    # Create a new DataFrame by concatenating the Series
    session_df = pd.DataFrame(series_list)
    
    os.makedirs(folder_path, exist_ok=True)  # Create folder if it doesn't exist
    filename = os.path.join(folder_path, f'S{session_id}_{start_date}_rawDataRight_{subject}.txt')
    session_df.to_csv(filename, sep=',', header=False, index=False)

print("--- %s seconds ---" % (time.time() - start_time))
for index, session in filtered_left_insole.iterrows():
    session_id = session['SESSION_ID']
    print(f'Session {session_id}')
    start_date = str(session['START_DATE'])
    date_folder = start_date[:4] + '-' + start_date[4:6] + '-' + start_date[6:]
    array_data = session.drop(['SESSION_ID', 'START_DATE', 'RAW_ID'])
    session_df = pd.DataFrame(array_data).T
    folder_path = os.path.join(root_folder_path, date_folder)
    
    # Convert each array to pandas Series
    series_list = {col: pd.Series(data) for col, data in array_data.items()}
    if len(series_list['READING_1']) < 2 / 0.05: #! data can be less than 2 seconds, which is not necessary
        print(f'Session {session_id} less than 2 seconds, Skip')
        print("-----------------------------------------------------------")
        continue
    # Create a new DataFrame by concatenating the Series
    session_df = pd.DataFrame(series_list)
    
    os.makedirs(folder_path, exist_ok=True)  # Create folder if it doesn't exist
    filename = os.path.join(folder_path, f'S{session_id}_{start_date}_rawDataLeft_{subject}.txt')
    session_df.to_csv(filename, sep=',', header=False, index=False)
    
print("--- Completed in %s seconds ---" % (time.time() - start_time))

--- 0.0 seconds ---
--- Read 0.002999544143676758 seconds ---
--- Merged in 0.010999441146850586 seconds ---
--- Filtered in 0.013998985290527344 seconds ---
Session 38 less than 2 seconds, Skip
-----------------------------------------------------------
--- 48.14009165763855 seconds ---
Session 1
Session 2
Session 3
Session 4
Session 5
Session 6
Session 7
Session 8
Session 9
Session 10
Session 11
Session 12
Session 13
Session 14
Session 15
Session 16
Session 17
Session 18
Session 19
Session 20
Session 21
Session 22
Session 23
Session 24
Session 25
Session 26
Session 27
Session 28
Session 29
Session 30
Session 31
Session 32
Session 33
Session 34
Session 35
Session 36
Session 37
Session 38
Session 38 less than 2 seconds, Skip
-----------------------------------------------------------
Session 39
Session 40
Session 41
Session 42
Session 43
Session 44
Session 45
Session 46
Session 47
Session 48
Session 49
Session 50
Session 51
Session 52
Session 53
Session 54
Session 55
Session 56
Session